<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/gradient_boosting_with_full_data_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir

files = listdir('/content/drive/MyDrive/amex-default-prediction/data/raw/')

In [2]:
import gc 

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [3]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
TRAIN_LABELS_PATH = '/content/train_labels.csv'

In [4]:
!unzip /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip \
 train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip
replace train_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_labels.csv        


In [5]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [6]:
def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [7]:
flag = True
X = pd.DataFrame()
Y = pd.DataFrame()

for f in files:
  print(f)
  df_tmp = pd.read_parquet('/content/drive/MyDrive/amex-default-prediction/data/raw/' + f)
  df_tmp = append_label(df_tmp)


  if(flag):
     enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
     df_tmp[['D_63', 'D_64']]=enc.fit(df_tmp[['D_63', 'D_64']])
     flag = False

  df_tmp[['D_63', 'D_64']]=enc.transform(df_tmp[['D_63', 'D_64']])
  df_tmp.fillna(0, inplace=True)

  X= X.append(df_tmp.loc[:, ~df_tmp.columns.isin(['customer_ID', 'S_2', 'target'])])

  Y = Y.append(df_tmp.loc[:, df_tmp.columns == 'target'])

  
  del df_tmp
  gc.collect()


data_2017-04.parquet
data_2017-05.parquet
data_2017-06.parquet
data_2017-03.parquet
data_2017-07.parquet
data_2017-08.parquet
data_2017-09.parquet
data_2017-10.parquet
data_2017-11.parquet
data_2017-12.parquet
data_2018-01.parquet
data_2018-02.parquet
data_2018-03.parquet


In [8]:
gc.collect()

58

In [9]:
 X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
gc.collect()

310

In [11]:
del X, Y

In [12]:
gc.collect()

176

In [13]:
clf = GradientBoostingClassifier(subsample=0.05, verbose=1)

In [14]:
clf.fit(X_train, y_train.values.ravel())

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.0434           0.0790           39.61m
         2           0.9827           0.0622           39.08m
         3           0.9308           0.0499           38.56m
         4           0.8911           0.0417           38.13m
         5           0.8556           0.0354           37.73m
         6           0.8236           0.0293           37.31m
         7           0.7985           0.0259           36.90m
         8           0.7764           0.0216           36.47m
         9           0.7584           0.0192           36.07m
        10           0.7411           0.0171           35.70m
        20           0.6506           0.0057           31.72m
        30           0.6138           0.0024           27.83m
        40           0.5946           0.0014           23.82m
        50           0.5821           0.0008           19.87m
        60           0.5754           0.0006           15.88m
       

GradientBoostingClassifier(subsample=0.05, verbose=1)

In [15]:
clf.score(X_test, y_test)

0.8734835590274168

In [16]:
clf.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 0.05,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 1,
 'warm_start': False}

In [17]:
y_pred = clf.predict(X_test)

In [18]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [19]:
tn, fp, fn, tp

(761525, 69600, 70364, 204802)

In [20]:
f1_score(y_test, y_pred)

0.7453199604052636